In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import logging
from environmental_risk_metrics import EsaLandCover, EsriLandCover, OpenLandMapLandCover
import geopandas as gpd

# Configure logging to display in Jupyter
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    force=True,  # This ensures the configuration is applied even if logging was previously configured
)
logger = logging.getLogger(__name__)



polygon = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates": [
            [
                [10.235198982658801, 51.42076009745068],
                [10.236477278753114, 51.41697045550828],
                [10.244461712820623, 51.41823370440062],
                [10.242888425319222, 51.4220355049745],
                [10.235198982658801, 51.42076009745068],
            ]
        ],
        "type": "Polygon",
    },
}
gdf = gpd.GeoDataFrame.from_features([polygon, polygon], crs="EPSG:4326")
start_date = "2000-01-01"
end_date = "2025-12-31"


esa_land_cover = EsaLandCover(gdf=gdf)
openlandmap_land_cover = OpenLandMapLandCover(gdf=gdf)
esri_land_cover = EsriLandCover(gdf=gdf)


esa_land_cover_data = esa_land_cover.get_data(
    start_date=start_date,
    end_date=end_date,
)

openlandmap_land_cover_data = openlandmap_land_cover.get_data(
    start_date=start_date,
    end_date=end_date,
)

esri_land_cover_data = esri_land_cover.get_data(
    start_date=start_date,
    end_date=end_date,
)

AttributeError: 'OpenLandMapLandCover' object has no attribute 'load_xarray'

In [2]:
data = esri_land_cover.get_items(start_date=start_date, end_date=end_date, polygon=gdf, polygon_crs=gdf.crs)

In [4]:
esa_land_cover_data

[{2000: {'Crops': 24.43,
   'No Data': 0,
   'Water': 0,
   'Trees': 0,
   'Flooded vegetation': 0,
   'Built area': 0,
   'Bare ground': 0,
   'Snow/ice': 0,
   'Clouds': 0,
   'Rangeland': 0},
  2001: {'Crops': 24.43,
   'No Data': 0,
   'Water': 0,
   'Trees': 0,
   'Flooded vegetation': 0,
   'Built area': 0,
   'Bare ground': 0,
   'Snow/ice': 0,
   'Clouds': 0,
   'Rangeland': 0},
  2002: {'Crops': 24.43,
   'No Data': 0,
   'Water': 0,
   'Trees': 0,
   'Flooded vegetation': 0,
   'Built area': 0,
   'Bare ground': 0,
   'Snow/ice': 0,
   'Clouds': 0,
   'Rangeland': 0},
  2003: {'Crops': 24.43,
   'No Data': 0,
   'Water': 0,
   'Trees': 0,
   'Flooded vegetation': 0,
   'Built area': 0,
   'Bare ground': 0,
   'Snow/ice': 0,
   'Clouds': 0,
   'Rangeland': 0},
  2004: {'Crops': 24.43,
   'No Data': 0,
   'Water': 0,
   'Trees': 0,
   'Flooded vegetation': 0,
   'Built area': 0,
   'Bare ground': 0,
   'Snow/ice': 0,
   'Clouds': 0,
   'Rangeland': 0},
  2005: {'Crops': 24.43,


In [3]:
esri_land_cover_data

[{'2023': {'Crops': 24.42},
  '2022': {'Crops': 24.42},
  '2021': {'Crops': 24.42},
  '2020': {'Crops': 24.42},
  '2019': {'Crops': 24.42},
  '2018': {'Crops': 24.42},
  '2017': {'Crops': 24.42}},
 {'2023': {'Crops': 24.42},
  '2022': {'Crops': 24.42},
  '2021': {'Crops': 24.42},
  '2020': {'Crops': 24.42},
  '2019': {'Crops': 24.42},
  '2018': {'Crops': 24.42},
  '2017': {'Crops': 24.42}}]

In [6]:
esri_land_cover.get_items(start_date=start_date, end_date=end_date, polygon=gdf, polygon_crs=gdf.crs)

In [ ]:
esa_data = esa_land_cover.load_xarray(start_date=start_date, end_date=end_date)

100%|██████████| 21/21 [00:00<00:00, 477.13it/s]


In [6]:
data = esa_land_cover.get_items(start_date=start_date, end_date=end_date, polygon=polygon, polygon_crs="EPSG:4326")

In [9]:
raster_path = data[0].assets["lccs_class"].href

with rasterio.open(raster_path) as src:
    resolution = src.res
    crs = src.crs
    # Calculate pixel area in hectares directly
    pixel_area_ha = abs(resolution[0] * resolution[1]) / 10000

In [14]:
from rasterstats import zonal_stats
import rasterio

# Get the raster file path
raster_path = data[0].assets["lccs_class"].href

# Open raster and get basic info
with rasterio.open(raster_path) as src:
    resolution = src.res
    crs = src.crs
    # Calculate pixel area in hectares directly
    pixel_area_ha = abs(resolution[0] * resolution[1]) / 10000

# Count pixels per land cover class within each polygon
stats_dict = zonal_stats(
    gdf,
    raster_path, 
    categorical=True,
    all_touched=False  # Only count pixels with centers inside polygon
)

# Convert pixel counts to hectares and scale to match expected area
expected_total_ha = 24.4226
areas_dict = []

for polygon_stats in stats_dict:
    total_pixels = sum(polygon_stats.values())
    scaling_factor = expected_total_ha / (total_pixels * pixel_area_ha)
    
    # Calculate area in hectares for each class
    area_stats = {
        class_id: count * pixel_area_ha * scaling_factor 
        for class_id, count in polygon_stats.items()
    }
    areas_dict.append(area_stats)


(0.002777777777777778, 0.0027777777777777783)
EPSG:4326


In [12]:
crs

CRS.from_wkt('GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]')

In [5]:
from rasterstats import zonal_stats
import rasterio

# Get the raster file path
raster_path = data[0].assets["lccs_class"].href

# Open raster and get basic info
with rasterio.open(raster_path) as src:
    resolution = src.res
    crs = src.crs
    # Calculate pixel area in hectares directly
    pixel_area_ha = abs(resolution[0] * resolution[1]) / 10000

# Count pixels per land cover class within each polygon
stats_dict = zonal_stats(
    gdf,
    raster_path, 
    categorical=True,
    all_touched=False  # Only count pixels with centers inside polygon
)

# Convert pixel counts to hectares and scale to match expected area
expected_total_ha = 24.4226
areas_dict = []

for polygon_stats in stats_dict:
    total_pixels = sum(polygon_stats.values())
    scaling_factor = expected_total_ha / (total_pixels * pixel_area_ha)
    
    # Calculate area in hectares for each class
    area_stats = {
        class_id: count * pixel_area_ha * scaling_factor 
        for class_id, count in polygon_stats.items()
    }
    areas_dict.append(area_stats)


NameError: name 'data' is not defined

In [56]:
244226 / 10000

24.4226

In [58]:
areas_dict

[{11: 20.352166666666665, 30: 4.070433333333333},
 {11: 20.352166666666665, 30: 4.070433333333333}]

In [29]:
stats_dict

[{11: 5, 30: 1}, {11: 5, 30: 1}]

In [9]:
# Get the data
ds_list = esa_land_cover.load_xarray(
    start_date=start_date,
    end_date=end_date,
)

area_list = []
for ds, polygon in zip(ds_list, gdf["geometry"]):
    # First clip the data in its native CRS (4326)
    clipped_data = ds.rio.clip(
        [polygon], crs=gdf.crs, all_touched=True
    )
    clipped_data = clipped_data.where(clipped_data[esa_land_cover.band_name] != 0)
    
    # Now reproject to equal-area projection for accurate area calculation
    # Mask again with the polygon in 6933
    polygon_proj = gpd.GeoSeries([polygon], crs=gdf.crs).to_crs(clipped_data.rio.crs).iloc[0]
    clipped_data_projected = clipped_data.rio.clip([polygon_proj], crs="EPSG:6933", all_touched=True)
    
    # Get pixel area from the projected data
    res = clipped_data_projected.rio.resolution()[0]
    pixel_area_sqm = res

    clipped_data_df = clipped_data_projected.to_dataframe()
    clipped_data_df[esa_land_cover.band_name] = clipped_data_df[esa_land_cover.band_name].map(
        esa_land_cover.get_legend_labels_dict()
    )
    grouped = clipped_data_df.groupby("time")
    value_counts = grouped[esa_land_cover.band_name].value_counts()

    area = value_counts * pixel_area_sqm
    area_list.append(round(area.unstack(level=1), 2))

data = area_list



100%|██████████| 21/21 [00:00<00:00, 474.90it/s]


NoDataInBounds: No data found in bounds. Data variable: lccs_class

In [8]:
data[0]

lccs_class,Crops
time,
2000-01-01,2127.0
2001-01-01,2127.0
2002-01-01,2127.0
2003-01-01,2127.0
2004-01-01,2127.0
2005-01-01,2127.0
2006-01-01,2127.0
2007-01-01,2127.0
2008-01-01,2127.0


In [4]:
data = esa_land_cover.load_xarray(start_date=start_date, end_date=end_date)

100%|██████████| 21/21 [00:00<00:00, 424.20it/s]


In [6]:
data[0].rio.crs

CRS.from_wkt('PROJCS["WGS 84 / UTM zone 32N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",9],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32632"]]')

In [8]:
754021.18/30

25134.039333333334

In [3]:
esri_land_cover_data

[[{'date': Timestamp('2017-01-01 00:00:00'),
   'Crops': 720000.0,
   'No Data': 270000.0,
   'Trees': 90000.0,
   'Water': 0,
   'Flooded vegetation': 0,
   'Built area': 0,
   'Bare ground': 0,
   'Snow/ice': 0,
   'Clouds': 0,
   'Rangeland': 0},
  {'date': Timestamp('2018-01-01 00:00:00'),
   'Crops': 810000.0,
   'No Data': 270000.0,
   'Trees': 0.0,
   'Water': 0,
   'Flooded vegetation': 0,
   'Built area': 0,
   'Bare ground': 0,
   'Snow/ice': 0,
   'Clouds': 0,
   'Rangeland': 0},
  {'date': Timestamp('2019-01-01 00:00:00'),
   'Crops': 810000.0,
   'No Data': 270000.0,
   'Trees': 0.0,
   'Water': 0,
   'Flooded vegetation': 0,
   'Built area': 0,
   'Bare ground': 0,
   'Snow/ice': 0,
   'Clouds': 0,
   'Rangeland': 0},
  {'date': Timestamp('2020-01-01 00:00:00'),
   'Crops': 810000.0,
   'No Data': 270000.0,
   'Trees': 0.0,
   'Water': 0,
   'Flooded vegetation': 0,
   'Built area': 0,
   'Bare ground': 0,
   'Snow/ice': 0,
   'Clouds': 0,
   'Rangeland': 0},
  {'date': T

In [3]:
esri_land_cover_data

[[{'date': Timestamp('2017-01-01 00:00:00'),
   'Crops': 256042.16,
   'No Data': 19080.68,
   'Water': 0,
   'Trees': 0,
   'Flooded vegetation': 0,
   'Built area': 0,
   'Bare ground': 0,
   'Snow/ice': 0,
   'Clouds': 0,
   'Rangeland': 0},
  {'date': Timestamp('2018-01-01 00:00:00'),
   'Crops': 256042.16,
   'No Data': 19080.68,
   'Water': 0,
   'Trees': 0,
   'Flooded vegetation': 0,
   'Built area': 0,
   'Bare ground': 0,
   'Snow/ice': 0,
   'Clouds': 0,
   'Rangeland': 0},
  {'date': Timestamp('2019-01-01 00:00:00'),
   'Crops': 256042.16,
   'No Data': 19080.68,
   'Water': 0,
   'Trees': 0,
   'Flooded vegetation': 0,
   'Built area': 0,
   'Bare ground': 0,
   'Snow/ice': 0,
   'Clouds': 0,
   'Rangeland': 0},
  {'date': Timestamp('2020-01-01 00:00:00'),
   'Crops': 256042.16,
   'No Data': 19080.68,
   'Water': 0,
   'Trees': 0,
   'Flooded vegetation': 0,
   'Built area': 0,
   'Bare ground': 0,
   'Snow/ice': 0,
   'Clouds': 0,
   'Rangeland': 0},
  {'date': Timestamp

In [14]:
data

[[{'date': Timestamp('2000-01-01 00:00:00'),
   'Crops': 100.0,
   'Crops_sqm': 720000.0,
   'No Data': 0,
   'No Data_sqm': 0,
   'Rangeland': 0,
   'Rangeland_sqm': 0,
   'Trees': 0,
   'Trees_sqm': 0,
   'Flooded vegetation': 0,
   'Flooded vegetation_sqm': 0,
   'Built area': 0,
   'Built area_sqm': 0,
   'Bare ground': 0,
   'Bare ground_sqm': 0,
   'Water': 0,
   'Water_sqm': 0,
   'Snow/ice': 0,
   'Snow/ice_sqm': 0},
  {'date': Timestamp('2001-01-01 00:00:00'),
   'Crops': 100.0,
   'Crops_sqm': 720000.0,
   'No Data': 0,
   'No Data_sqm': 0,
   'Rangeland': 0,
   'Rangeland_sqm': 0,
   'Trees': 0,
   'Trees_sqm': 0,
   'Flooded vegetation': 0,
   'Flooded vegetation_sqm': 0,
   'Built area': 0,
   'Built area_sqm': 0,
   'Bare ground': 0,
   'Bare ground_sqm': 0,
   'Water': 0,
   'Water_sqm': 0,
   'Snow/ice': 0,
   'Snow/ice_sqm': 0},
  {'date': Timestamp('2002-01-01 00:00:00'),
   'Crops': 100.0,
   'Crops_sqm': 720000.0,
   'No Data': 0,
   'No Data_sqm': 0,
   'Rangeland'

In [9]:
esri_land_cover = EsriLandCover(gdf=gdf, use_esri_classes=True)

esri = esri_land_cover.load_xarray(
    start_date=start_date,
    end_date=end_date,
)

esri_percentages = esri_land_cover.get_land_use_class_percentages(
    start_date=start_date,
    end_date=end_date,
)

esri_percentages

100%|██████████| 7/7 [00:00<00:00, 90.96it/s]


[data        Crops
 time             
 2017-01-01  100.0
 2018-01-01  100.0
 2019-01-01  100.0
 2020-01-01  100.0
 2021-01-01  100.0
 2022-01-01  100.0
 2023-01-01  100.0,
 data        Crops
 time             
 2017-01-01  100.0
 2018-01-01  100.0
 2019-01-01  100.0
 2020-01-01  100.0
 2021-01-01  100.0
 2022-01-01  100.0
 2023-01-01  100.0]

In [12]:

olanm_land_cover = OpenLandMapLandCover(gdf=gdf, use_esri_classes=True)

olanm = olanm_land_cover.load_xarray(
    start_date=start_date,
    end_date=end_date,
)
olanm

2025-01-21 17:13:20,914 - rasterio._env - INFO - GDAL signalled an error: err_no=4, msg="`/vsicurl/https://s3.openlandmap.org/arco/lc_glad.glcluc_c_30m_s_20000101_20001231_go_epsg.4326_v20230901.tif' does not exist in the file system, and is not recognized as a supported dataset name."


RasterioIOError: '/vsicurl/https://s3.openlandmap.org/arco/lc_glad.glcluc_c_30m_s_20000101_20001231_go_epsg.4326_v20230901.tif' does not exist in the file system, and is not recognized as a supported dataset name.

In [14]:
olanm_land_cover.get_land_use_class_percentages(
    start_date=start_date,
    end_date=end_date,
)

2025-01-21 17:13:27,866 - rasterio._env - INFO - GDAL signalled an error: err_no=4, msg="`/vsicurl/https://s3.openlandmap.org/arco/lc_glad.glcluc_c_30m_s_20000101_20001231_go_epsg.4326_v20230901.tif' does not exist in the file system, and is not recognized as a supported dataset name."


RasterioIOError: '/vsicurl/https://s3.openlandmap.org/arco/lc_glad.glcluc_c_30m_s_20000101_20001231_go_epsg.4326_v20230901.tif' does not exist in the file system, and is not recognized as a supported dataset name.

In [17]:



luc = esa_land_cover.load_xarray(
    start_date=start_date,
    end_date=end_date,
)

class_percentages = esa_land_cover.get_land_use_class_percentages(
    start_date=start_date,
    end_date=end_date,
)
class_percentages


100%|██████████| 21/21 [00:00<00:00, 126.69it/s]


[lccs_class  Crops
 time             
 2000-01-01  100.0
 2001-01-01  100.0
 2002-01-01  100.0
 2003-01-01  100.0
 2004-01-01  100.0
 2005-01-01  100.0
 2006-01-01  100.0
 2007-01-01  100.0
 2008-01-01  100.0
 2009-01-01  100.0
 2010-01-01  100.0
 2011-01-01  100.0
 2012-01-01  100.0
 2013-01-01  100.0
 2014-01-01  100.0
 2015-01-01  100.0
 2016-01-01  100.0
 2017-01-01  100.0
 2018-01-01  100.0
 2019-01-01  100.0
 2020-01-01  100.0,
 lccs_class  Crops
 time             
 2000-01-01  100.0
 2001-01-01  100.0
 2002-01-01  100.0
 2003-01-01  100.0
 2004-01-01  100.0
 2005-01-01  100.0
 2006-01-01  100.0
 2007-01-01  100.0
 2008-01-01  100.0
 2009-01-01  100.0
 2010-01-01  100.0
 2011-01-01  100.0
 2012-01-01  100.0
 2013-01-01  100.0
 2014-01-01  100.0
 2015-01-01  100.0
 2016-01-01  100.0
 2017-01-01  100.0
 2018-01-01  100.0
 2019-01-01  100.0
 2020-01-01  100.0]

In [18]:
olanm_land_cover = OpenLandMapLandCover(use_esri_classes=True)

olanm_percentages = olanm_land_cover.get_land_use_class_percentages(
    start_date=start_date,
    end_date=end_date,
    polygon=polygon,
    polygon_crs="EPSG:4326",
)
olanm_percentages


TypeError: OpenLandMapLandCover.__init__() missing 1 required positional argument: 'gdf'

In [6]:
olanm_land_cover = OpenLandMapLandCover(use_esri_classes=True)
olanm_land_cover.create_map(
    polygons=polygon,
    polygon_crs="EPSG:4326",
)

Map(center=[np.float64(10.239830347739712), np.float64(51.41950298024139)], controls=(ZoomControl(options=['po…